In [1]:
import pandas as pd
import numpy as np

In [2]:

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:

from sklearn.ensemble import RandomForestRegressor


In [6]:
# Load the Train Parquet file for EDA
raw_train_df = pd.read_parquet("../datasets/raw_train_oct_2024.snappy.parquet")

print("Dataset loaded successfully!")
raw_train_df.head()  # View first few rows


Dataset loaded successfully!


,stock_item_id,last_date_seen,first_date_seen,days_to_sell,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,...,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
0,b6598141a3d1dca4825a72e0f55ddfeb0fb55cabbc14da...,2024-10-12,2024-10-09,3,3695,3695,False,1.0,Independent,5.0,...,2010-09-23,"1 LADY OWNER FROM NEW, 2 KEYS",False,GREAT,3897.0,87353.0,16,FRONT_RIGHT,62,L
1,acdde4604ab932227b68abff979123a3f0b5d26d8b711e...,2024-10-11,2024-09-13,28,18998,18298,False,6.3,Franchise,5.0,...,2021-07-31,Sales event now on.,False,FAIR,17268.0,31423.0,19,FRONT_RIGHT,46,B
2,0d29437822e059c038fc7f6385d9c348ba3727509add9a...,2024-10-05,2024-08-23,43,13990,13990,False,0.2,Independent,5.0,...,2021-06-29,1 OWNER | SAT NAV | DAB |,False,LOW,15316.0,23882.0,28,FRONT_RIGHT,70,PE
3,bbd82c7e47cba47f96ac59c7022ff1ca485c869f660778...,2024-10-15,2024-09-30,15,8174,7990,False,0.3,Independent,4.0,...,2016-03-23,"8 SERVICE RECORDS, SAT NAV",False,GOOD,8085.0,58342.0,24,FRONT_RIGHT,52,LE
4,2515373efa5d36e0220cd42cc1cea23686e547cad17ca3...,2024-10-07,2024-08-15,53,9450,8750,False,7.3,Independent,5.0,...,2014-09-10,"ULEZ COMPLIANT, 6M WARRANTY",False,LOW,9529.0,69003.0,34,FRONT_LEFT,47,SS


In [7]:
raw_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112086 entries, 0 to 112085
Data columns (total 43 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   stock_item_id                                    112086 non-null  object 
 1   last_date_seen                                   112086 non-null  object 
 2   first_date_seen                                  112086 non-null  object 
 3   days_to_sell                                     112086 non-null  int32  
 4   first_retailer_asking_price                      112086 non-null  int64  
 5   last_retailer_asking_price                       112086 non-null  int64  
 6   can_home_deliver                                 112086 non-null  bool   
 7   reviews_per_100_advertised_stock_last_12_months  101795 non-null  float64
 8   segment                                          112018 non-null  object 
 9   seats          

In [8]:
# Sample 30% of the data (replace 0.3 with your desired fraction)
sampled_df = raw_train_df.sample(frac=0.1, random_state=42)

# View the sampled data
sampled_df.head()

,stock_item_id,last_date_seen,first_date_seen,days_to_sell,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,...,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
46997,5f64f3d07c551eafb23ab72716889c3c32994726b4dd8a...,2024-10-28,2024-10-11,17,1995,1995,False,4.0,Independent,5.0,...,2010-03-30,NATIONWIDE DELIVERY AVAILABLE,False,GREAT,2147.0,67776.0,24,FRONT_LEFT,41,HG
12633,e3a8155d7705608ff97154a4646f66ac758f74af014f6f...,2024-10-16,2024-10-03,13,15290,15090,False,18.9,Independent,5.0,...,2018-02-12,CLICK&COLLECT/HOME DELIVERY,False,GOOD,14802.0,53429.0,40,FRONT_RIGHT,65,DE
13800,46dac54a3c356997106fd7771b4a54140f86b68d4134fb...,2024-10-12,2024-09-13,29,24395,24395,False,12.5,Franchise,5.0,...,2023-01-30,⭐FREE 12M WARRANTY & ROADSIDE⭐,True,FAIR,23342.0,12266.0,34,FRONT_RIGHT,71,CA
58091,cece0b30dd92bbb869a82cf4963f9c8a5c979cb45a8aaf...,2024-10-19,2024-08-24,56,9500,9580,False,4.6,Franchise,5.0,...,2019-01-31,360°camera-Sat nav-BOSE audio,False,FAIR,9269.0,35836.0,39,FRONT_RIGHT,38,SR
44513,02ad65a65f89c2d9bcababc6d3285dc99ca3c0d86acaf7...,2024-10-26,2024-10-22,4,309000,309000,False,28.7,Franchise,2.0,...,2021-11-01,None,True,NOANALYSIS,NaN,4851.0,28,FRONT_RIGHT,67,NG


In [9]:
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11209 entries, 46997 to 77515
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   stock_item_id                                    11209 non-null  object 
 1   last_date_seen                                   11209 non-null  object 
 2   first_date_seen                                  11209 non-null  object 
 3   days_to_sell                                     11209 non-null  int32  
 4   first_retailer_asking_price                      11209 non-null  int64  
 5   last_retailer_asking_price                       11209 non-null  int64  
 6   can_home_deliver                                 11209 non-null  bool   
 7   reviews_per_100_advertised_stock_last_12_months  10164 non-null  float64
 8   segment                                          11206 non-null  object 
 9   seats                        

In [10]:
X = sampled_df.drop(columns=['days_to_sell'])  # Drop target from features
y = sampled_df['days_to_sell']  # Target variable

## Transformation Step to Replace 'None' with 'np.Nan'

In [153]:
# Replace None with Nan Transformer.
def replace_none_with_nan(df: pd.DataFrame) -> pd.DataFrame:
    """Replaces None with np.nan in categorical columns."""
    df = df.copy()
    categorical_cols = df.select_dtypes(include=['object']).columns
    df[categorical_cols] = df[categorical_cols].apply(lambda col: col.map(lambda x: np.nan if x is None else x))
    return df

replace_none_transformer = FunctionTransformer(replace_none_with_nan, validate=False)

# test transformer
replace_none_transformer.fit_transform(X)

,stock_item_id,last_date_seen,first_date_seen,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,doors,...,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
46997,5f64f3d07c551eafb23ab72716889c3c32994726b4dd8a...,2024-10-28,2024-10-11,1995,1995,False,4.0,Independent,5.0,5.0,...,2010-03-30,NATIONWIDE DELIVERY AVAILABLE,False,GREAT,2147.0,67776.0,24,FRONT_LEFT,41,HG
12633,e3a8155d7705608ff97154a4646f66ac758f74af014f6f...,2024-10-16,2024-10-03,15290,15090,False,18.9,Independent,5.0,5.0,...,2018-02-12,CLICK&COLLECT/HOME DELIVERY,False,GOOD,14802.0,53429.0,40,FRONT_RIGHT,65,DE
13800,46dac54a3c356997106fd7771b4a54140f86b68d4134fb...,2024-10-12,2024-09-13,24395,24395,False,12.5,Franchise,5.0,5.0,...,2023-01-30,⭐FREE 12M WARRANTY & ROADSIDE⭐,True,FAIR,23342.0,12266.0,34,FRONT_RIGHT,71,CA
58091,cece0b30dd92bbb869a82cf4963f9c8a5c979cb45a8aaf...,2024-10-19,2024-08-24,9500,9580,False,4.6,Franchise,5.0,5.0,...,2019-01-31,360°camera-Sat nav-BOSE audio,False,FAIR,9269.0,35836.0,39,FRONT_RIGHT,38,SR
44513,02ad65a65f89c2d9bcababc6d3285dc99ca3c0d86acaf7...,2024-10-26,2024-10-22,309000,309000,False,28.7,Franchise,2.0,2.0,...,2021-11-01,NaN,True,NOANALYSIS,NaN,4851.0,28,FRONT_RIGHT,67,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44878,7726e8523ce6b42b97271f6da307d8d07be2168e10c17f...,2024-10-27,2024-09-13,12310,12770,False,13.6,Franchise,5.0,5.0,...,2021-09-30,SAT NAV/CAR PLAY/HEATED SEATS,False,GOOD,12550.0,18881.0,40,FRONT_RIGHT,44,NG
50179,6724a492b1c218b5d86993bfc6b852a15b8a8b994e2dac...,2024-10-18,2024-09-13,28795,27850,False,0.3,Franchise,5.0,5.0,...,2021-09-01,ACTIVE CRUISE CONTROL!,False,GOOD,27355.0,30356.0,36,FRONT_RIGHT,41,GU
77419,f9634c97097ad2e0c2fbd53e39047bd09020aa9aa58606...,2024-10-06,2024-09-19,25498,25500,False,5.3,Franchise,5.0,5.0,...,2022-10-28,Part-exchange offer now on,False,FAIR,24307.0,17262.0,19,PROMOTIONAL_MATERIAL,36,G
46793,62a6a19b17b826af8c8bab24e2ed5cf8e2232c7dc3e2b9...,2024-10-04,2024-09-13,23499,23499,False,29.2,Franchise,5.0,5.0,...,2022-09-26,NaN,False,FAIR,22099.0,11616.0,40,FRONT_RIGHT,49,TS


## Transformation Step to Drop Certain Categorical features containing dates and ids

In [154]:
def drop_cat_features(df: pd.DataFrame) -> pd.DataFrame:
    """Drop features not required"""
    df = df.copy()
    columns_to_drop = ["stock_item_id", "last_date_seen", "first_date_seen", "derivative_id", "first_registration_date"]
    return df.drop(columns=columns_to_drop, errors='ignore')

drop_cat_features_transformer = FunctionTransformer(drop_cat_features)

drop_cat_features_transformer.fit_transform(X)

,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,doors,co2_emission_gpkm,top_speed_mph,zero_to_sixty_mph_seconds,...,odometer_reading_miles,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
46997,1995,1995,False,4.0,Independent,5.0,5.0,154.0,101.0,13.7,...,79897,NATIONWIDE DELIVERY AVAILABLE,False,GREAT,2147.0,67776.0,24,FRONT_LEFT,41,HG
12633,15290,15090,False,18.9,Independent,5.0,5.0,149.0,155.0,6.5,...,42166,CLICK&COLLECT/HOME DELIVERY,False,GOOD,14802.0,53429.0,40,FRONT_RIGHT,65,DE
13800,24395,24395,False,12.5,Franchise,5.0,5.0,148.0,146.0,None,...,15974,⭐FREE 12M WARRANTY & ROADSIDE⭐,True,FAIR,23342.0,12266.0,34,FRONT_RIGHT,71,CA
58091,9500,9580,False,4.6,Franchise,5.0,5.0,162.0,111.0,None,...,48846,360°camera-Sat nav-BOSE audio,False,FAIR,9269.0,35836.0,39,FRONT_RIGHT,38,SR
44513,309000,309000,False,28.7,Franchise,2.0,2.0,154.0,211.0,2.5,...,2716,None,True,NOANALYSIS,NaN,4851.0,28,FRONT_RIGHT,67,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44878,12310,12770,False,13.6,Franchise,5.0,5.0,119.0,121.0,9.3,...,23339,SAT NAV/CAR PLAY/HEATED SEATS,False,GOOD,12550.0,18881.0,40,FRONT_RIGHT,44,NG
50179,28795,27850,False,0.3,Franchise,5.0,5.0,0.0,112.0,None,...,46471,ACTIVE CRUISE CONTROL!,False,GOOD,27355.0,30356.0,36,FRONT_RIGHT,41,GU
77419,25498,25500,False,5.3,Franchise,5.0,5.0,153.0,125.0,None,...,18211,Part-exchange offer now on,False,FAIR,24307.0,17262.0,19,PROMOTIONAL_MATERIAL,36,G
46793,23499,23499,False,29.2,Franchise,5.0,5.0,109.0,107.0,None,...,7555,None,False,FAIR,22099.0,11616.0,40,FRONT_RIGHT,49,TS


## Transformation Step to force convert Categorical Features to Numerical Features

In [155]:
def convert_columns_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Converts specified object columns to float/numeric type without dropping them."""
    df = df.copy()
    
    convert_columns = ["zero_to_sixty_mph_seconds", "engine_power_bhp", "fuel_economy_wltp_combined_mpg",
                       "battery_usable_capacity_kwh", "length_mm", "insurance_group", "plate"]
    
    # Ensure we only select columns that exist in the DataFrame
    existing_columns = list(set(df.columns) & set(convert_columns))
    
    if existing_columns:  # Only apply conversion if columns exist
        df[existing_columns] = df[existing_columns].apply(pd.to_numeric, errors='coerce')

    return df

# Example: Wrap it in a FunctionTransformer
convert_to_numeric_transformer = FunctionTransformer(convert_columns_to_numeric)

# Apply the transformer
convert_to_numeric_transformer.fit_transform(X)

,stock_item_id,last_date_seen,first_date_seen,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,doors,...,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
46997,5f64f3d07c551eafb23ab72716889c3c32994726b4dd8a...,2024-10-28,2024-10-11,1995,1995,False,4.0,Independent,5.0,5.0,...,2010-03-30,NATIONWIDE DELIVERY AVAILABLE,False,GREAT,2147.0,67776.0,24,FRONT_LEFT,41,HG
12633,e3a8155d7705608ff97154a4646f66ac758f74af014f6f...,2024-10-16,2024-10-03,15290,15090,False,18.9,Independent,5.0,5.0,...,2018-02-12,CLICK&COLLECT/HOME DELIVERY,False,GOOD,14802.0,53429.0,40,FRONT_RIGHT,65,DE
13800,46dac54a3c356997106fd7771b4a54140f86b68d4134fb...,2024-10-12,2024-09-13,24395,24395,False,12.5,Franchise,5.0,5.0,...,2023-01-30,⭐FREE 12M WARRANTY & ROADSIDE⭐,True,FAIR,23342.0,12266.0,34,FRONT_RIGHT,71,CA
58091,cece0b30dd92bbb869a82cf4963f9c8a5c979cb45a8aaf...,2024-10-19,2024-08-24,9500,9580,False,4.6,Franchise,5.0,5.0,...,2019-01-31,360°camera-Sat nav-BOSE audio,False,FAIR,9269.0,35836.0,39,FRONT_RIGHT,38,SR
44513,02ad65a65f89c2d9bcababc6d3285dc99ca3c0d86acaf7...,2024-10-26,2024-10-22,309000,309000,False,28.7,Franchise,2.0,2.0,...,2021-11-01,None,True,NOANALYSIS,NaN,4851.0,28,FRONT_RIGHT,67,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44878,7726e8523ce6b42b97271f6da307d8d07be2168e10c17f...,2024-10-27,2024-09-13,12310,12770,False,13.6,Franchise,5.0,5.0,...,2021-09-30,SAT NAV/CAR PLAY/HEATED SEATS,False,GOOD,12550.0,18881.0,40,FRONT_RIGHT,44,NG
50179,6724a492b1c218b5d86993bfc6b852a15b8a8b994e2dac...,2024-10-18,2024-09-13,28795,27850,False,0.3,Franchise,5.0,5.0,...,2021-09-01,ACTIVE CRUISE CONTROL!,False,GOOD,27355.0,30356.0,36,FRONT_RIGHT,41,GU
77419,f9634c97097ad2e0c2fbd53e39047bd09020aa9aa58606...,2024-10-06,2024-09-19,25498,25500,False,5.3,Franchise,5.0,5.0,...,2022-10-28,Part-exchange offer now on,False,FAIR,24307.0,17262.0,19,PROMOTIONAL_MATERIAL,36,G
46793,62a6a19b17b826af8c8bab24e2ed5cf8e2232c7dc3e2b9...,2024-10-04,2024-09-13,23499,23499,False,29.2,Franchise,5.0,5.0,...,2022-09-26,None,False,FAIR,22099.0,11616.0,40,FRONT_RIGHT,49,TS


In [156]:
convert_to_numeric_transformer.fit_transform(X).info()

<class 'pandas.core.frame.DataFrame'>
Index: 11209 entries, 46997 to 77515
Data columns (total 42 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   stock_item_id                                    11209 non-null  object 
 1   last_date_seen                                   11209 non-null  object 
 2   first_date_seen                                  11209 non-null  object 
 3   first_retailer_asking_price                      11209 non-null  int64  
 4   last_retailer_asking_price                       11209 non-null  int64  
 5   can_home_deliver                                 11209 non-null  bool   
 6   reviews_per_100_advertised_stock_last_12_months  10164 non-null  float64
 7   segment                                          11206 non-null  object 
 8   seats                                            11203 non-null  float64
 9   doors                        

## Complete Preprocessing Transformation Pipeline

In [157]:
# Define Categorical pipeline

data_preparation_pipeline = Pipeline([
    
    ('drop_columns', drop_cat_features_transformer), # Drop id and date columns
    ('replace_none', replace_none_transformer),  # Custom transformer to handle 'None'
    ('convert_2numeric', convert_to_numeric_transformer)
])

transformed_raw_df = data_preparation_pipeline.fit_transform(X)
transformed_raw_df

,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,doors,co2_emission_gpkm,top_speed_mph,zero_to_sixty_mph_seconds,...,odometer_reading_miles,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
46997,1995,1995,False,4.0,Independent,5.0,5.0,154.0,101.0,13.7,...,79897,NATIONWIDE DELIVERY AVAILABLE,False,GREAT,2147.0,67776.0,24,FRONT_LEFT,41,HG
12633,15290,15090,False,18.9,Independent,5.0,5.0,149.0,155.0,6.5,...,42166,CLICK&COLLECT/HOME DELIVERY,False,GOOD,14802.0,53429.0,40,FRONT_RIGHT,65,DE
13800,24395,24395,False,12.5,Franchise,5.0,5.0,148.0,146.0,NaN,...,15974,⭐FREE 12M WARRANTY & ROADSIDE⭐,True,FAIR,23342.0,12266.0,34,FRONT_RIGHT,71,CA
58091,9500,9580,False,4.6,Franchise,5.0,5.0,162.0,111.0,NaN,...,48846,360°camera-Sat nav-BOSE audio,False,FAIR,9269.0,35836.0,39,FRONT_RIGHT,38,SR
44513,309000,309000,False,28.7,Franchise,2.0,2.0,154.0,211.0,2.5,...,2716,NaN,True,NOANALYSIS,NaN,4851.0,28,FRONT_RIGHT,67,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44878,12310,12770,False,13.6,Franchise,5.0,5.0,119.0,121.0,9.3,...,23339,SAT NAV/CAR PLAY/HEATED SEATS,False,GOOD,12550.0,18881.0,40,FRONT_RIGHT,44,NG
50179,28795,27850,False,0.3,Franchise,5.0,5.0,0.0,112.0,NaN,...,46471,ACTIVE CRUISE CONTROL!,False,GOOD,27355.0,30356.0,36,FRONT_RIGHT,41,GU
77419,25498,25500,False,5.3,Franchise,5.0,5.0,153.0,125.0,NaN,...,18211,Part-exchange offer now on,False,FAIR,24307.0,17262.0,19,PROMOTIONAL_MATERIAL,36,G
46793,23499,23499,False,29.2,Franchise,5.0,5.0,109.0,107.0,NaN,...,7555,NaN,False,FAIR,22099.0,11616.0,40,FRONT_RIGHT,49,TS


In [158]:
data_preparation_pipeline

Pipeline(steps=[('drop_columns',
                 FunctionTransformer(func=<function drop_cat_features at 0x729f06cde700>)),
                ('replace_none',
                 FunctionTransformer(func=<function replace_none_with_nan at 0x729f380b4400>)),
                ('convert_2numeric',
                 FunctionTransformer(func=<function convert_columns_to_numeric at 0x729f06cdefc0>))])

In [159]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11209 entries, 46997 to 77515
Data columns (total 42 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   stock_item_id                                    11209 non-null  object 
 1   last_date_seen                                   11209 non-null  object 
 2   first_date_seen                                  11209 non-null  object 
 3   first_retailer_asking_price                      11209 non-null  int64  
 4   last_retailer_asking_price                       11209 non-null  int64  
 5   can_home_deliver                                 11209 non-null  bool   
 6   reviews_per_100_advertised_stock_last_12_months  10164 non-null  float64
 7   segment                                          11206 non-null  object 
 8   seats                                            11203 non-null  float64
 9   doors                        

In [160]:
categorical_df = X.select_dtypes(include=["object", "bool"])
categorical_df

,stock_item_id,last_date_seen,first_date_seen,can_home_deliver,segment,zero_to_sixty_mph_seconds,engine_power_bhp,fuel_economy_wltp_combined_mpg,battery_usable_capacity_kwh,length_mm,...,fuel_type,transmission_type,drivetrain,colour,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,first_image_label,postcode_area
46997,5f64f3d07c551eafb23ab72716889c3c32994726b4dd8a...,2024-10-28,2024-10-11,False,Independent,13.7,79.0,43.5,None,4020.0,...,Petrol,Manual,Front Wheel Drive,Silver,2010-03-30,NATIONWIDE DELIVERY AVAILABLE,False,GREAT,FRONT_LEFT,HG
12633,e3a8155d7705608ff97154a4646f66ac758f74af014f6f...,2024-10-16,2024-10-03,False,Independent,6.5,227.0,43.5,None,4670.0,...,Petrol,Manual,Front Wheel Drive,Grey,2018-02-12,CLICK&COLLECT/HOME DELIVERY,False,GOOD,FRONT_RIGHT,DE
13800,46dac54a3c356997106fd7771b4a54140f86b68d4134fb...,2024-10-12,2024-09-13,False,Franchise,None,221.0,43.7,None,4400.0,...,Petrol,Automatic,Front Wheel Drive,White,2023-01-30,⭐FREE 12M WARRANTY & ROADSIDE⭐,True,FAIR,FRONT_RIGHT,CA
58091,cece0b30dd92bbb869a82cf4963f9c8a5c979cb45a8aaf...,2024-10-19,2024-08-24,False,Franchise,None,110.0,34.0,None,4135.0,...,Petrol,Manual,Front Wheel Drive,Grey,2019-01-31,360°camera-Sat nav-BOSE audio,False,FAIR,FRONT_RIGHT,SR
44513,02ad65a65f89c2d9bcababc6d3285dc99ca3c0d86acaf7...,2024-10-26,2024-10-22,False,Franchise,2.5,972.0,46.31,6.4,4710.0,...,Petrol Plug-in Hybrid,Automatic,Four Wheel Drive,Red,2021-11-01,None,True,NOANALYSIS,FRONT_RIGHT,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44878,7726e8523ce6b42b97271f6da307d8d07be2168e10c17f...,2024-10-27,2024-09-13,False,Franchise,9.3,99.0,52.3,None,4060.0,...,Petrol,Manual,Front Wheel Drive,Black,2021-09-30,SAT NAV/CAR PLAY/HEATED SEATS,False,GOOD,FRONT_RIGHT,NG
50179,6724a492b1c218b5d86993bfc6b852a15b8a8b994e2dac...,2024-10-18,2024-09-13,False,Franchise,None,282.0,None,73.8,4734.0,...,Electric,Automatic,Rear Wheel Drive,Blue,2021-09-01,ACTIVE CRUISE CONTROL!,False,GOOD,FRONT_RIGHT,GU
77419,f9634c97097ad2e0c2fbd53e39047bd09020aa9aa58606...,2024-10-06,2024-09-19,False,Franchise,None,163.0,41.5,None,4575.0,...,Petrol,Manual,Front Wheel Drive,Red,2022-10-28,Part-exchange offer now on,False,FAIR,PROMOTIONAL_MATERIAL,G
46793,62a6a19b17b826af8c8bab24e2ed5cf8e2232c7dc3e2b9...,2024-10-04,2024-09-13,False,Franchise,None,108.0,58.9,None,4090.0,...,Petrol Hybrid,Automatic,Front Wheel Drive,Blue,2022-09-26,None,False,FAIR,FRONT_RIGHT,TS


In [161]:
numeric_df = X.select_dtypes(include=[np.number])
numeric_df

,first_retailer_asking_price,last_retailer_asking_price,reviews_per_100_advertised_stock_last_12_months,seats,doors,co2_emission_gpkm,top_speed_mph,battery_range_miles,boot_space_seats_up_litres,odometer_reading_miles,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,advert_quality
46997,1995,1995,4.0,5.0,5.0,154.0,101.0,NaN,337.0,79897,2147.0,67776.0,24,41
12633,15290,15090,18.9,5.0,5.0,149.0,155.0,NaN,590.0,42166,14802.0,53429.0,40,65
13800,24395,24395,12.5,5.0,5.0,148.0,146.0,NaN,430.0,15974,23342.0,12266.0,34,71
58091,9500,9580,4.6,5.0,5.0,162.0,111.0,NaN,354.0,48846,9269.0,35836.0,39,38
44513,309000,309000,28.7,2.0,2.0,154.0,211.0,15.0,74.0,2716,NaN,4851.0,28,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44878,12310,12770,13.6,5.0,5.0,119.0,121.0,NaN,309.0,23339,12550.0,18881.0,40,44
50179,28795,27850,0.3,5.0,5.0,0.0,112.0,282.0,510.0,46471,27355.0,30356.0,36,41
77419,25498,25500,5.3,5.0,5.0,153.0,125.0,NaN,522.0,18211,24307.0,17262.0,19,36
46793,23499,23499,29.2,5.0,5.0,109.0,107.0,NaN,298.0,7555,22099.0,11616.0,40,49


In [162]:
# Identify numeric and categorical columns
numeric_features = numeric_df.columns
categorical_features = categorical_df.columns

In [163]:


# Numerical pipeline
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
    ('scaler', StandardScaler())  # Scale numeric features
])

# Categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values for categorical
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine into a ColumnTransformer to handle both types
preprocessor = ColumnTransformer([
    ('numeric_features', numeric_pipeline, numeric_features),
    #('categorical_features', categorical_pipeline, categorical_features)
])


preprocessor

ColumnTransformer(transformers=[('numeric_features',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['first_retailer_asking_price', 'last_retailer_asking_price',
       'reviews_per_100_advertised_stock_last_12_months', 'seats', 'doors',
       'co2_emission_gpkm', 'top_speed_mph', 'battery_range_miles',
       'boot_space_seats_up_litres', 'odometer_reading_miles',
       'adjusted_retail_amount_gbp', 'predicted_mileage', 'number_of_images',
       'advert_quality'],
      dtype='object'))])

In [164]:
preprocessor.fit_transform(X)

array([[-0.82221673, -0.8534527 , -0.56982368, ...,  0.91160737,
        -0.61133111, -0.85896075],
       [-0.10505842, -0.09887447,  0.92931553, ...,  0.37020569,
         0.35747277,  0.74617811],
       [ 0.38608309,  0.43731114,  0.28538996, ..., -1.18313071,
        -0.00582868,  1.14746283],
       ...,
       [ 0.44558107,  0.50098498, -0.4390263 , ..., -0.99460051,
        -0.91408232, -1.19336468],
       [ 0.3377511 ,  0.38568059,  1.96563324, ..., -1.20765926,
         0.35747277, -0.32391446],
       [ 0.42184661,  0.41789206, -0.12712485, ..., -0.18493198,
        -0.06637893,  0.41177418]], shape=(11209, 14))

In [165]:
# Final pipeline combining the preprocessor and classifier
final_pipeline = Pipeline([
    ('data_preparation', data_preparation_pipeline),
    ('data_preprocessor', preprocessor),
    ('model_regressor', RandomForestRegressor())
])

final_pipeline

Pipeline(steps=[('data_preparation',
                 Pipeline(steps=[('drop_columns',
                                  FunctionTransformer(func=<function drop_cat_features at 0x729f06cde700>)),
                                 ('replace_none',
                                  FunctionTransformer(func=<function replace_none_with_nan at 0x729f380b4400>)),
                                 ('convert_2numeric',
                                  FunctionTransformer(func=<function convert_columns_to_numeric at 0x729f06cdefc0>))])),
                ('data_preprocess...
                                                  Index(['first_retailer_asking_price', 'last_retailer_asking_price',
       'reviews_per_100_advertised_stock_last_12_months', 'seats', 'doors',
       'co2_emission_gpkm', 'top_speed_mph', 'battery_range_miles',
       'boot_space_seats_up_litres', 'odometer_reading_miles',
       'adjusted_retail_amount_gbp', 'predicted_mileage', 'number_of_images',
       'advert_quality'],
      dtype='object'))])),
                ('model_regressor', RandomForestRegressor())])

In [166]:
#final_pipeline.fit_transform(raw_train_df)

In [167]:
final_pipeline.fit(X, y)

Pipeline(steps=[('data_preparation',
                 Pipeline(steps=[('drop_columns',
                                  FunctionTransformer(func=<function drop_cat_features at 0x729f06cde700>)),
                                 ('replace_none',
                                  FunctionTransformer(func=<function replace_none_with_nan at 0x729f380b4400>)),
                                 ('convert_2numeric',
                                  FunctionTransformer(func=<function convert_columns_to_numeric at 0x729f06cdefc0>))])),
                ('data_preprocess...
                                                  Index(['first_retailer_asking_price', 'last_retailer_asking_price',
       'reviews_per_100_advertised_stock_last_12_months', 'seats', 'doors',
       'co2_emission_gpkm', 'top_speed_mph', 'battery_range_miles',
       'boot_space_seats_up_litres', 'odometer_reading_miles',
       'adjusted_retail_amount_gbp', 'predicted_mileage', 'number_of_images',
       'advert_quality'],
      dtype='object'))])),
                ('model_regressor', RandomForestRegressor())])

In [168]:
# Use the score() method on the entire pipeline
r2_score = final_pipeline.score(X, y)

print(f"R² score: {r2_score}")

R² score: 0.8875977965845359


The presence of certain values represented as 'None' in categorical data columns makes dataframe that should originally be numerical to be representated as cateforical instead. 
Data processing will be performed on these data columns transformining them back to numerical features

In [27]:
# # Retrieve columns containing 'None' values
# columns_with_none = categorical_df.columns[categorical_df.apply(
#     lambda col: col.map(lambda x: x == None).any())]

# print(columns_with_none, len(columns_with_none))

In [58]:
raw_train_df

,stock_item_id,last_date_seen,first_date_seen,days_to_sell,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,...,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
0,b6598141a3d1dca4825a72e0f55ddfeb0fb55cabbc14da...,2024-10-12,2024-10-09,3,3695,3695,False,1.0,Independent,5.0,...,2010-09-23,"1 LADY OWNER FROM NEW, 2 KEYS",False,GREAT,3897.0,87353.0,16,FRONT_RIGHT,62,L
1,acdde4604ab932227b68abff979123a3f0b5d26d8b711e...,2024-10-11,2024-09-13,28,18998,18298,False,6.3,Franchise,5.0,...,2021-07-31,Sales event now on.,False,FAIR,17268.0,31423.0,19,FRONT_RIGHT,46,B
2,0d29437822e059c038fc7f6385d9c348ba3727509add9a...,2024-10-05,2024-08-23,43,13990,13990,False,0.2,Independent,5.0,...,2021-06-29,1 OWNER | SAT NAV | DAB |,False,LOW,15316.0,23882.0,28,FRONT_RIGHT,70,PE
3,bbd82c7e47cba47f96ac59c7022ff1ca485c869f660778...,2024-10-15,2024-09-30,15,8174,7990,False,0.3,Independent,4.0,...,2016-03-23,"8 SERVICE RECORDS, SAT NAV",False,GOOD,8085.0,58342.0,24,FRONT_RIGHT,52,LE
4,2515373efa5d36e0220cd42cc1cea23686e547cad17ca3...,2024-10-07,2024-08-15,53,9450,8750,False,7.3,Independent,5.0,...,2014-09-10,"ULEZ COMPLIANT, 6M WARRANTY",False,LOW,9529.0,69003.0,34,FRONT_LEFT,47,SS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112081,972aa9a8f7b3c4b010a61898c55809252cc9e7f1f4a1a2...,2024-10-15,2024-09-27,18,5995,5995,False,8.7,Independent,5.0,...,2015-03-25,FREE TAX,False,FAIR,5777.0,59567.0,17,FRONT_LEFT,54,G
112082,7197de1e741d66a3356dfbf863dad033211d42f96ce316...,2024-10-29,2024-07-26,95,33198,31299,False,9.2,Independent,5.0,...,2022-09-01,LEATHER | HEATED SEATS,False,LOW,33838.0,17129.0,42,FRONT_RIGHT,59,E
112083,467f841cb64f99ba2bc1dbcd5d33b8d19c02a69a034240...,2024-10-19,2024-07-18,93,48950,46450,False,NaN,Franchise,5.0,...,2024-01-18,Comfort & Technology Pack,True,GOOD,46794.0,8312.0,46,FRONT_RIGHT,57,BT
112084,7cebcc19bff375c2666cf2d59a8d0d07f70aebad308410...,2024-10-12,2024-08-16,57,1850,1699,False,5.1,Independent,5.0,...,2007-03-03,12 months mot free warranty,False,NOANALYSIS,NaN,NaN,23,FRONT_LEFT,51,M


In [83]:
transformed_df = cat_pipeline.fit_transform(categorical_df)

# # Convert to DataFrame if necessary (some transformers return NumPy arrays)
# if isinstance(transformed_df, np.ndarray):
#     transformed_df = pd.DataFrame(transformed_df, columns=df.columns)

# Select numeric columns3


## Select Column Types

### Select Numeric DataFrame

In [11]:
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112086 entries, 0 to 112085
Data columns (total 15 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   days_to_sell                                     112086 non-null  int32  
 1   first_retailer_asking_price                      112086 non-null  int64  
 2   last_retailer_asking_price                       112086 non-null  int64  
 3   reviews_per_100_advertised_stock_last_12_months  101795 non-null  float64
 4   seats                                            112023 non-null  float64
 5   doors                                            112080 non-null  float64
 6   co2_emission_gpkm                                112025 non-null  float64
 7   top_speed_mph                                    111806 non-null  float64
 8   battery_range_miles                              7789 non-null    float64
 9   boot_space_seat

In [ ]:
numeric_df = raw_train_df.select_dtypes(include=[np.number])
numeric_df